In [1]:
import warnings
warnings.simplefilter("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
import gensim
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.initializers import Constant
from keras.layers import (Input, Embedding, SpatialDropout1D, Bidirectional,
                          GlobalAveragePooling1D, GlobalMaxPooling1D,
                          LSTM, CuDNNLSTM, GRU, CuDNNGRU, concatenate, Dropout, Dense, Activation)
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
NUM_WORDS = 50000
MAXLEN = 250
EMBEDDING_DIM = 300

In [3]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch + 1, score))

In [4]:
train = pd.read_csv('data/train_preprocessed.csv')
test = pd.read_csv('data/test_preprocessed.csv')
train['comment_text'].fillna("", inplace=True)
test['comment_text'].fillna("", inplace=True)
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
X_train_raw, y_train = train['comment_text'].str.lower(), train[classes].values
X_test_raw = test['comment_text'].str.lower()
tk = Tokenizer(num_words=NUM_WORDS)
tk.fit_on_texts(X_train_raw)
X_train = tk.texts_to_sequences(X_train_raw)
X_test = tk.texts_to_sequences(X_test_raw)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
X_train = pad_sequences(X_train, maxlen=MAXLEN)
X_valid = pad_sequences(X_valid, maxlen=MAXLEN)
X_test = pad_sequences(X_test, maxlen=MAXLEN)
nb_words = min(NUM_WORDS, len(tk.word_index) + 1)

In [5]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def get_embedding(pretrained_word_vectors):
    if pretrained_word_vectors == "google":
        EMBEDDING_PATH = "../embeddings/GoogleNews-vectors-negative300.bin"
        embedding_index = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_PATH, binary=True)
    if pretrained_word_vectors == "glove":
        EMBEDDING_PATH = "../embeddings/glove.840B.300d.txt"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    elif pretrained_word_vectors == "fasttext":
        EMBEDDING_PATH = "../embeddings/crawl-300d-2M.vec"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in tk.word_index.items():
        if i >= NUM_WORDS:
            continue
        if pretrained_word_vectors == "google":
            try:
                embedding_vector = embedding_index.get_vector(word)
            except Exception:
                continue
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [6]:
def build_model_LSTM(embedding_matrix):
    inputs = Input(shape=(MAXLEN,))
    x = Embedding(nb_words, EMBEDDING_DIM,
                  embeddings_initializer=Constant(embedding_matrix),
                  trainable=False)(inputs)
    avg_pool_x = GlobalAveragePooling1D()(x)
    max_pool_x = GlobalMaxPooling1D()(x)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    avg_pool_h1 = GlobalAveragePooling1D()(x)
    max_pool_h1 = GlobalMaxPooling1D()(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    avg_pool_h2 = GlobalAveragePooling1D()(x)
    max_pool_h2 = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool_x, max_pool_x,
                     avg_pool_h1, max_pool_h1,
                     avg_pool_h2, max_pool_h2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [7]:
embedding_matrix = get_embedding("glove")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model_LSTM(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v5_MultiBiLSTM_glove_preprocess_submission.csv.gz", compression="gzip", index=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 93s 650us/step - loss: 0.0593 - acc: 0.9787 - val_loss: 0.0421 - val_acc: 0.9840

 ROC-AUC - epoch: 1 - score: 0.983486
Epoch 2/10
143613/143613 [==============================] - 96s 668us/step - loss: 0.0443 - acc: 0.9827 - val_loss: 0.0405 - val_acc: 0.9845

 ROC-AUC - epoch: 2 - score: 0.986926
Epoch 3/10
143613/143613 [==============================] - 94s 655us/step - loss: 0.0414 - acc: 0.9838 - val_loss: 0.0392 - val_acc: 0.9847

 ROC-AUC - epoch: 3 - score: 0.989297
Epoch 4/10
143613/143613 [==============================] - 93s 648us/step - loss: 0.0392 - acc:

In [8]:
embedding_matrix = get_embedding("fasttext")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model_LSTM(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v5_MultiBiLSTM_fasttext_preprocess_submission.csv.gz", compression="gzip", index=False)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 87s 606us/step - loss: 0.0583 - acc: 0.9793 - val_loss: 0.0432 - val_acc: 0.9831

 ROC-AUC - epoch: 1 - score: 0.984006
Epoch 2/10
143613/143613 [==============================] - 89s 623us/step - loss: 0.0440 - acc: 0.9828 - val_loss: 0.0400 - val_acc: 0.9844

 ROC-AUC - epoch: 2 - score: 0.988424
Epoch 3/10
143613/143613 [==============================] - 90s 625us/step - loss: 0.0413 - acc: 0.9836 - val_loss: 0.0390 - val_acc: 0.9846

 ROC-AUC - epoch: 3 - score: 0.989723
Epoch 4/10
143613/143613 [==============================] - 88s 615us/step - loss: 0.0389 - acc: 0.9844 - val_loss: 0.0421 - val_acc: 0.9847

 ROC-AUC - epoch: 4 - score: 0.990223
Epoch 5/10
143613/143613 [==============================] - 89s 618us/step - loss: 0.0374 - acc: 0.9849 - val_loss: 0.0406 - val_acc: 0.9850

 ROC-AUC - epoch: 5 - score: 0.990013
Epoch 6/10
143613/143613 [=======================

In [9]:
def build_model_GRU(embedding_matrix):
    inputs = Input(shape=(MAXLEN,))
    x = Embedding(nb_words, EMBEDDING_DIM,
                  embeddings_initializer=Constant(embedding_matrix),
                  trainable=False)(inputs)
    avg_pool_x = GlobalAveragePooling1D()(x)
    max_pool_x = GlobalMaxPooling1D()(x)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    avg_pool_h1 = GlobalAveragePooling1D()(x)
    max_pool_h1 = GlobalMaxPooling1D()(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    avg_pool_h2 = GlobalAveragePooling1D()(x)
    max_pool_h2 = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool_x, max_pool_x,
                     avg_pool_h1, max_pool_h1,
                     avg_pool_h2, max_pool_h2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [10]:
embedding_matrix = get_embedding("glove")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model_GRU(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v5_MultiBiGRU_glove_preprocess_submission.csv.gz", compression="gzip", index=False)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 81s 564us/step - loss: 0.0564 - acc: 0.9799 - val_loss: 0.0415 - val_acc: 0.9841

 ROC-AUC - epoch: 1 - score: 0.984703
Epoch 2/10
143613/143613 [==============================] - 84s 584us/step - loss: 0.0439 - acc: 0.9830 - val_loss: 0.0405 - val_acc: 0.9844

 ROC-AUC - epoch: 2 - score: 0.987905
Epoch 3/10
143613/143613 [==============================] - 84s 583us/step - loss: 0.0409 - acc: 0.9838 - val_loss: 0.0419 - val_acc: 0.9839

 ROC-AUC - epoch: 3 - score: 0.988214
Epoch 4/10
143613/143613 [==============================] - 84s 585us/step - loss: 0.0388 - acc: 0.9844 - val_loss: 0.0388 - val_acc: 0.9848

 ROC-AUC - epoch: 4 - score: 0.990194
Epoch 5/10
143613/143613 [==============================] - 84s 582us/step - loss: 0.0370 - acc: 0.9851 - val_loss: 0.0384 - val_acc: 0.9847

 ROC-AUC - epoch: 5 - score: 0.990594
Epoch 6/10
143613/143613 [=======================

In [11]:
embedding_matrix = get_embedding("fasttext")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model_GRU(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v5_MultiBiGRU_fasttext_preprocess_submission.csv.gz", compression="gzip", index=False)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 80s 559us/step - loss: 0.0595 - acc: 0.9788 - val_loss: 0.0450 - val_acc: 0.9828

 ROC-AUC - epoch: 1 - score: 0.983466
Epoch 2/10
143613/143613 [==============================] - 83s 579us/step - loss: 0.0453 - acc: 0.9825 - val_loss: 0.0412 - val_acc: 0.9833

 ROC-AUC - epoch: 2 - score: 0.986175
Epoch 3/10
143613/143613 [==============================] - 84s 584us/step - loss: 0.0425 - acc: 0.9832 - val_loss: 0.0401 - val_acc: 0.9843

 ROC-AUC - epoch: 3 - score: 0.988047
Epoch 4/10
143613/143613 [==============================] - 84s 583us/step - loss: 0.0404 - acc: 0.9839 - val_loss: 0.0394 - val_acc: 0.9845

 ROC-AUC - epoch: 4 - score: 0.989296
Epoch 5/10
143613/143613 [==============================] - 84s 586us/step - loss: 0.0387 - acc: 0.9845 - val_loss: 0.0389 - val_acc: 0.9848

 ROC-AUC - epoch: 5 - score: 0.989300
Epoch 6/10
143613/143613 [=======================